# DCA Ingest - DROID Analysis

Dieses Notebook führt eine DROID-Analyse auf einem Ordner durch und speichert die Ergebnisse als CSV.

## Imports und Setup

In [1]:
import os
import subprocess

## Konfiguration

Hier werden alle Pfade und Parameter definiert.

In [2]:
# Dynamisch das Home-Verzeichnis holen
home_dir = os.path.expanduser("~")
base_path = os.path.join(home_dir, "work")

# Dataset-Name definieren
dataset_to_analyze = "2104_mockup"

# Pfade definieren
droid_script_path = os.path.join(base_path, "27_DCA_Ingest/src/droid-binary-6.7.0-bin/droid.sh")
folder_to_analyze = os.path.join(base_path, f"dcaonnextcloud-500gb/DigitalMaterialCopies/Semiramis/302_V-Zug_Documentation/{dataset_to_analyze}")
output_folder = os.path.join(base_path, f"dca-metadataraw/Semiramis/{dataset_to_analyze}_results")
output_csv_path = os.path.join(output_folder, "analysis_result_2.csv")

print(f"Analyzing folder: {folder_to_analyze}")
print(f"Output will be saved to: {output_csv_path}")

Analyzing folder: /home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/Semiramis/302_V-Zug_Documentation/2104_mockup
Output will be saved to: /home/renku/work/dca-metadataraw/Semiramis/2104_mockup_results/analysis_result_2.csv


## DROID-Analyse ausführen

Führt die DROID-Analyse auf dem konfigurierten Ordner aus.

In [3]:
try:
    # Output-Ordner erstellen (falls nicht vorhanden)
    os.makedirs(output_folder, exist_ok=True)

    print(f"Analyzing folder: {folder_to_analyze}")
    print(f"Output will be saved to: {output_csv_path}")

    # DROID-Skript ausführen
    result = subprocess.run(
        [droid_script_path, "-R", folder_to_analyze, "-o", output_csv_path, "-Pr", "profile.generateHash=true"],
        check=True,
        capture_output=True,
        text=True
    )

    print("DROID output:", result.stdout)
    print(f"Analysis complete. The result is saved in {output_csv_path}.")

except subprocess.CalledProcessError as e:
    print(f"An error occurred while running DROID: {e}")
    print("DROID error output:", e.stderr)

except PermissionError as e:
    print(f"Permission error: {e}")
    print("Bitte prüfe, ob du Schreibrechte für den Zielordner hast.")

except FileNotFoundError as e:
    print(f"Datei oder Verzeichnis nicht gefunden: {e}")
    print("Bitte prüfe, ob alle Pfade korrekt sind und existieren.")

Analyzing folder: /home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/Semiramis/302_V-Zug_Documentation/2104_mockup
Output will be saved to: /home/renku/work/dca-metadataraw/Semiramis/2104_mockup_results/analysis_result_2.csv
DROID output: 2025-11-12T12:59:47,216  INFO [main] DroidCommandLine:225 - Starting DROID.
2025-11-12T12:59:48,406  INFO [main] ProfileManagerImpl:129 - Creating profile: 1762952388406
2025-11-12T12:59:48,425  INFO [main] ProfileInstance:365 - Attempting state change [INITIALISING] to [VIRGIN]
2025-11-12T12:59:49,848 ERROR [main] WriterResultHandlerDao:172 - /home/renku/work/dca-metadataraw/Semiramis/2104_mockup_results/analysis_result_2.csv (Read-only file system)
2025-11-12T12:59:51,854  INFO [main] ProfileDiskAction:87 - Saving profile [/home/renku/.droid6/profiles/1762952388406/db] to [/home/renku/.droid6/profile_templates/schema 6.7.0/profile.template]
2025-11-12T12:59:56,380  INFO [main] ProfileDiskAction:87 - Saving profile [/home/renku/.droid6/profi

## Analyse der DROID-Ergebnisse

In diesem Abschnitt werden die DROID-Ergebnisse analysiert und visualisiert.

In [ ]:
# Imports für die Analyse
import sys
import pandas as pd
import matplotlib.pyplot as plt

# Pfad zur src/ hinzufügen, um lokale Module zu importieren
src_path = os.path.join(base_path, "27_DCA_Ingest/src")
if src_path not in sys.path:
    sys.path.append(src_path)

# Optionale Installation von pywaffle (nur ausführen wenn nötig)
# !pip install pywaffle

print(f"sys.path erweitert um: {src_path}")

In [ ]:
# CSV-Pfad und Output-Verzeichnis für die Analyse konfigurieren
# Verwende die bereits im Notebook definierten Variablen
CSV_PATH = output_csv_path
OUTPUT_DIR = os.path.join(output_folder, 'res')

# Output-Verzeichnis erstellen falls nötig
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Analyse-CSV: {CSV_PATH}")
print(f"Ausgabe-Verzeichnis: {OUTPUT_DIR}")

In [ ]:
# Analyse der Dateiformate
try:
    from analyse_formats import analyse_formats
    
    analyse_formats(
        csv_path=CSV_PATH,
        output_dir=OUTPUT_DIR,
        show_plot=False  # Im Notebook-Kontext nicht anzeigen
    )
except ImportError as e:
    print(f"Hinweis: Modul 'analyse_formats' konnte nicht importiert werden: {e}")
    print("Bitte stelle sicher, dass sich das Modul im src-Verzeichnis befindet.")
except Exception as e:
    print(f"Fehler bei der Ausführung von analyse_formats: {e}")

In [ ]:
# Analyse eines spezifischen Formats über die Zeit
try:
    from analyse_format_over_time import analyse_format_over_time
    
    # Beispielformat - kann angepasst werden
    format_to_analyze = "Portable Document Format"  # Beispiel: PDF
    
    analyse_format_over_time(
        csv_path=CSV_PATH,
        format_name=format_to_analyze,
        output_dir=OUTPUT_DIR,
        show_plot=False
    )
except ImportError as e:
    print(f"Hinweis: Modul 'analyse_format_over_time' konnte nicht importiert werden: {e}")
except Exception as e:
    print(f"Fehler bei der Ausführung von analyse_format_over_time: {e}")

In [ ]:
# Waffle-Chart für Formatverteilung über Zeit
try:
    from generate_waffle import generate_waffle_chart
    
    # Parameter können angepasst werden
    generate_waffle_chart(
        csv_path=CSV_PATH,
        output_dir=OUTPUT_DIR,
        start_year=2004,  # Anpassen je nach Datensatz
        end_year=2009,    # Anpassen je nach Datensatz
        top_n=10,
        show_plot=False
    )
except ImportError as e:
    print(f"Hinweis: Modul 'generate_waffle' konnte nicht importiert werden: {e}")
    print("Falls pywaffle fehlt: pip install pywaffle")
except Exception as e:
    print(f"Fehler bei der Ausführung von generate_waffle_chart: {e}")

In [ ]:
# Format-Heatmap (mit try/except, da im Original manchmal fehlerhaft)
try:
    from analyse_format_heatmap import generate_format_heatmap
    
    generate_format_heatmap(
        csv_path=CSV_PATH,
        output_dir=OUTPUT_DIR,
        start_year=2004,  # Anpassen je nach Datensatz
        end_year=2009,    # Anpassen je nach Datensatz
        top_n=10,
        show_plot=False
    )
except ImportError as e:
    print(f"Hinweis: Modul 'analyse_format_heatmap' konnte nicht importiert werden: {e}")
except Exception as e:
    print(f"Fehler bei der Ausführung von generate_format_heatmap: {e}")